Import packages

In [28]:
import numpy as np
import pandas as pd

Data import

In [29]:
df = pd.read_excel("benchmarks.xlsx", sheet_name="data")

Data analysis

In [30]:
df.head(5)

,Currency,Name,RIC,30.09.2023,29.09.2023,28.09.2023,27.09.2023,26.09.2023,25.09.2023,24.09.2023,...,10.01.2021,09.01.2021,08.01.2021,07.01.2021,06.01.2021,05.01.2021,04.01.2021,03.01.2021,02.01.2021,01.01.2021
0,AUD,AUD ON deposit,AUDOND=,3.173120,3.173120,3.173120,3.173120,3.173120,3.173120,3.173120,...,-0.214400,-0.214400,-0.214400,-0.214400,-0.214400,-0.214400,-0.214400,-0.214400,-0.214400,-0.214400
1,BRL,Selic ON,BRSELICD=CBBR,10.848640,10.848640,10.848640,10.848640,10.848640,10.848640,10.848640,...,1.629440,1.629440,1.629440,1.629440,1.629440,1.629440,1.629440,0.000000,0.000000,0.000000
2,CHF,SARON,SARON.S,1.470692,1.470692,1.463197,1.459413,1.459776,1.459515,1.458327,...,-0.620393,-0.620393,-0.620393,-0.620811,-0.622444,-0.621344,-0.622502,0.000000,0.000000,0.000000
3,CNY,CNY ON deposit,CNYONID=CNR,1.523955,1.523955,1.523955,1.356723,1.351578,1.675750,1.707482,...,1.011110,1.011110,1.011110,0.990528,0.605466,0.571162,1.633728,8.582861,8.582861,8.582861
4,EUR,ESTER,EUROSTR=,3.327488,3.327488,3.349786,3.348928,3.348070,3.348070,3.348928,...,-0.481971,-0.481971,-0.481971,-0.484544,-0.485402,-0.482829,-0.485402,0.000000,0.000000,0.000000


In [31]:
df.tail(5)

,Currency,Name,RIC,30.09.2023,29.09.2023,28.09.2023,27.09.2023,26.09.2023,25.09.2023,24.09.2023,...,10.01.2021,09.01.2021,08.01.2021,07.01.2021,06.01.2021,05.01.2021,04.01.2021,03.01.2021,02.01.2021,01.01.2021
29,SEK,SWESTR,SWESTR=RIKS,3.342925,3.342925,3.353216,3.351501,3.125094,3.124237,3.125952,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000
30,SGD,SORA,SORA=MAST,3.279891,3.279891,3.107256,3.193016,3.308535,3.234953,3.225434,...,0.167232,0.167232,0.167232,0.151967,0.111145,0.102740,0.077270,0.000,0.000,0.000
31,THB,THOR,THONRP=BKTH,2.134052,2.134052,2.132663,2.137894,1.922816,1.925423,1.923820,...,0.424409,0.424409,0.424409,0.423534,0.423980,0.423046,0.416596,0.000,0.000,0.000
32,UAH,UONIA,UONIA=NBUK,13.587814,13.587814,13.721600,13.721600,13.721600,13.721600,13.721600,...,4.320160,4.320160,4.320160,4.320160,4.320160,4.341600,4.288000,0.000,0.000,0.000
33,ZAR,ZAR ON deposit,ZAROND=,5.145600,5.145600,5.917440,6.346240,6.260480,6.260480,6.088960,...,3.387520,3.387520,3.387520,3.173120,4.073600,4.288000,5.360000,4.288,4.288,4.288


In [32]:
df.shape

(34, 1006)

Data modelling

In [33]:
# Delete column 'Name'
df = df.drop("Name", axis=1)

In [34]:
# Unpivot all columns except 'Currency', 'RIC' -> melt columns containing date
df = pd.melt(df, id_vars=['Currency', 'RIC'], var_name='Date', value_name='Universal Close Price')

In [35]:
# Add empty column 'Universal Bid Price'
df['Universal Bid Price'] = np.nan

In [36]:
# Reorder columns
df = df[['Date', 'RIC', 'Currency', 'Universal Close Price', 'Universal Bid Price']]

# Rename columns
datascope_colnames = {'Date': 'Trade Date',
                      'Currency': 'Currency Code'}
df = df.rename(columns=datascope_colnames)

Create file per date

In [37]:
# Unique dates
date_sequence = df['Trade Date'].unique()
print('There are', len(date_sequence), 'date stamps')

There are 1003 date stamps


In [38]:
def get_filename(date):
    """
    Generate a filename based on the provided date.

    Args:
        date (str): A date string in the format "DD.MM.YYYY"

    Returns:
        str: A filename in the format "YYYYMMDD_on_ir_benchmark".

    Example:
        >>> get_filename('30.09.2023')
        '20230930_on_ir_benchmark'
    """
    date_list = date.split('.')[::-1]
    str_empty = ''
    prefix_date = str_empty.join(date_list)
    filename = f"{prefix_date}_on_ir_benchmark"
    return filename

In [39]:
# for loop to create an overnight benchmark file for each date
for date_item in date_sequence:
    # Get date
    df_subset = df[df['Trade Date'] == date_item]
    # Get file name & set path
    file_name = get_filename(date_item)
    file_name_path = f"files_created/{file_name}.csv"
    # Create a file
    df_subset.to_csv(file_name_path, index=None, sep=',', decimal=',')

Test: Show created file

In [40]:
df_test = pd.read_csv('files_created/20230930_on_ir_benchmark.csv')

In [41]:
df_test.head(5)

,Trade Date,RIC,Currency Code,Universal Close Price,Universal Bid Price
0,30.09.2023,AUDOND=,AUD,"3,1731200000000004",NaN
1,30.09.2023,BRSELICD=CBBR,BRL,"10,848640000000001",NaN
2,30.09.2023,SARON.S,CHF,"1,4706922368",NaN
3,30.09.2023,CNYONID=CNR,CNY,"1,5239552",NaN
4,30.09.2023,EUROSTR=,EUR,"3,3274880000000002",NaN
